In [1]:
import ee
import folium
from datetime import datetime, timedelta

In [2]:
# Initialize Earth Engine
ee.Initialize()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [3]:
#Define the indexes (Normalized Difference)
# Define the NBR function
def NBR(image):
    return image.expression('float(b("B8") - b("B12")) / float(b("B8") + b("B12"))')

# Define the NDMI function
def NDMI(image):
    return image.expression('float(b("B8") - b("B11")) / float(b("B8") + b("B11"))')

In [4]:
# Visualization parameters for layers
#vis_p = {'min': 0, 'max': 1, 'palette': ['green', 'yellow', 'orange', 'red']}
vis_p = {'min': 0, 'max': 1, 'palette': ['00FF00', 'FFFF00', 'FF9900', 'FF0000']}
layer_vis = {
    'nbr': vis_p,
    'ndmi': vis_p,
    'fire_risk': vis_p
}

In [5]:
# Calculate the last week's date range
today = datetime.today()
end_date = today.strftime('%Y-%m-%d')
start_date = (today - timedelta(days=7)).strftime('%Y-%m-%d')


# Define the region of interest (ROI)
lat = -19.307916656027874
lon = -59.116664414196585
#roi = ee.Geometry.Rectangle([lon - 0.3, lat - 0.3, lon + 0.3, lat + 0.3]) 

# Load Sentinel-2 collection and filter by date and ROI
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .select(['B8', 'B11', 'B12'])  # Bands for NBR and NDMI
    #.filterBounds(roi) \

In [6]:
# Calculate the mean NBR and NDMI
nbr_mean = collection.map(NBR).mean()
ndmi_mean = collection.map(NDMI).mean()

In [7]:
# Combine layers to create a fire risk map
fire_risk = nbr_mean.multiply(0.7).add(ndmi_mean.multiply(0.3))
#.clip(roi)

# Create a Folium map
m = folium.Map(location=[lat, lon], zoom_start=11)

In [8]:
#Displaying layers

# Add NBR layer
nbr_map = nbr_mean.getMapId(layer_vis['nbr'])
folium.raster_layers.TileLayer(
    tiles=nbr_map['tile_fetcher'].url_format,
    attr="Map Data &copy; <a href='https://earthengine.google.com'>Google Earth Engine</a>",
    name="NBR",
    overlay=True
).add_to(m)

# Add NDMI layer
ndmi_map = ndmi_mean.getMapId(layer_vis['ndmi'])
folium.raster_layers.TileLayer(
    tiles=ndmi_map['tile_fetcher'].url_format,
    attr="Map Data &copy; <a href='https://earthengine.google.com'>Google Earth Engine</a>",
    name="NDMI",
    overlay=True
).add_to(m)

# Add Fire Risk layer
fire_risk_map = fire_risk.getMapId(layer_vis['fire_risk'])
folium.raster_layers.TileLayer(
    tiles=fire_risk_map['tile_fetcher'].url_format,
    attr="Map Data &copy; <a href='https://earthengine.google.com'>Google Earth Engine</a>",
    name="Fire Risk",
    overlay=True
).add_to(m)


# Add layer control
folium.LayerControl().add_to(m)

# Display map
m

In [11]:
# Save the map to an HTML file
name="fire_risk_map(10-17nov).html"
m.save(name)
print("Fire Risk Map saved as", name)

Fire Risk Map saved as fire_risk_map(10-17nov).html
